Find-Stop-Headings
===========

To compute the stop-stop distance, we use the excellent [GraphHopper library](https://github.com/graphhopper/graphhopper).

However there is one prerequisite: we need to know the direction in which buses
arrive and depart at stops, otherwise, the routing algorithm might tell the bus
to go in the reverse direction.

OneMap and Data Mall provide a list of all Singapore bus stops. However, they
do not tell us the direction of the road that the bus stops are at. So we derive
this by:

1. Identify the public bus routes that use each bus stop
2. Identify bus stops preceding and succeeding each stop along each route
3. Route from the preceding stop to this stop, or this stop to succeeding stop
4. Take the first segment of the route path, and use that to impute the direction 
    of the road
5. (Verification) ensure that imputed directions are within, 30 degrees of each other.
    If they are more than 90 degrees apart, then we treat the direction as unknown
    
This will leave us with some stops whose direction are still unknown, either because
they are at an interchange, or because no buses call at the stop (usually bus stops
listed on OneMap but not Data Mall).

For these, we override the directions manually.

Running this notebook
-------------------------

Before running this notebook, please do the following:

### 1. Set up Graphhopper
execute the Docker image and expose port 8989:
```
$ docker build -t sg-graphhopper graphhopper
$ docker run --rm -p 8989:8989 -it sg-graphhopper
```

(Alternatively, if you have Java installed, just download the relevant JARs
and PBF files and get started. Refer to `graphhopper/Dockerfile` for more
information)

### 2. Download stops and routes data from OneMap and DataMall
You will also need to run the following scripts to download stops and routes from
DataMall and OneMap:

```
$ python fetch_bus_stops_from_onemap.py
$ python fetch_bus_stops_from_data_mall.py
$ python combine_onemap_and_data_mall.py
```

This notebook is also available as a script (`find_stop_headings.py`)

In [103]:
import requests
import json

In [2]:
BusRoutes = json.load(open('./bus_routes_from_data_mall.json'))

In [3]:
BusStops = json.load(open('./combined_bus_stops.json'))

In [42]:
def uniq_by(ls, key=lambda x: x):
    s = set()
    
    for item in ls:
        k = key(item)
        
        if k in s:
            continue
        else:
            s.add(k)
            yield item

def dedup_successive(ls):
    return [x for x,y in it.groupby(ls)]

In [43]:
import itertools as it

stopsByRoute = []

for (service, direction), route_stops in it.groupby(BusRoutes, lambda x : (x['ServiceNo'], x['Direction'])):
    sorted_route_stops = sorted(
        uniq_by(route_stops, key=lambda x:x['StopSequence']),
        key=lambda x:x['StopSequence']
    )
    
    stopsByRoute.append(((service, direction), sorted_route_stops))
    
filter(lambda x:x[0][0] == '410', stopsByRoute)

[((u'410', 1),
  [{u'BusStopCode': u'53009',
    u'Direction': 1,
    u'Distance': 0,
    u'Operator': u'SBST',
    u'SAT_FirstBus': u'0545',
    u'SAT_LastBus': u'0045',
    u'SUN_FirstBus': u'0545',
    u'SUN_LastBus': u'0045',
    u'ServiceNo': u'410',
    u'StopSequence': 1,
    u'WD_FirstBus': u'0545',
    u'WD_LastBus': u'0045'},
   {u'BusStopCode': u'53261',
    u'Direction': 1,
    u'Distance': 0.6,
    u'Operator': u'SBST',
    u'SAT_FirstBus': u'0548',
    u'SAT_LastBus': u'0048',
    u'SUN_FirstBus': u'0548',
    u'SUN_LastBus': u'0048',
    u'ServiceNo': u'410',
    u'StopSequence': 2,
    u'WD_FirstBus': u'0548',
    u'WD_LastBus': u'0048'},
   {u'BusStopCode': u'53391',
    u'Direction': 1,
    u'Distance': 1.2,
    u'Operator': u'SBST',
    u'SAT_FirstBus': u'0552',
    u'SAT_LastBus': u'0051',
    u'SUN_FirstBus': u'0550',
    u'SUN_LastBus': u'0050',
    u'ServiceNo': u'410',
    u'StopSequence': 3,
    u'WD_FirstBus': u'0550',
    u'WD_LastBus': u'0050'},
   {u'BusSto

In [44]:
# for each route, emit (this stop, next stop)

# obtain the (prev, next) pairs
def prev_next(bus_stops):
    for a,b in zip(bus_stops, bus_stops[1:]):
        if a['BusStopCode'] == b['BusStopCode']:
            raise ValueError((a['BusStopCode'], bus_stops))
            
        yield a['BusStopCode'], b['BusStopCode']

stop_pairs = [
    (prev, next)
    for sbr in stopsByRoute
    for prev, next in prev_next(sbr[1])
]
# Exclude the non-stopping ones
stop_pairs = [x for x in stop_pairs if not x[0].startswith('E') and not x[1].startswith('E')]
stop_pairs = list(set(stop_pairs))

stop_pairs[1]

(u'77139', u'77029')

In [45]:
filter(lambda (x,y): x == y, stop_pairs)

[]

In [46]:
import pyproj
svy = pyproj.Proj(init='epsg:3414')

In [47]:
def make_request(pair):
    ll1 = BusStops[pair[0]]
    ll2 = BusStops[pair[1]]
    
    return ['%s,%s' % (ll1['Latitude'], ll1['Longitude']),
            '%s,%s' % (ll2['Latitude'], ll2['Longitude'])]


def get_route(stop_pair):
    import polyline
    
    res = requests.get('http://localhost:8989/route',
        params={
            "point": [
                make_request(stop_pair)
            ],
            "type": "json",
            "vehicle": "car",
            "weighting": "fastest",
        })
    
    res = json.loads(res.text)
    
    if not 'paths' in res:
        return None
    
    return polyline.decode(res['paths'][0]['points'])

def has_stop_location(stop_pair):
    return stop_pair[0] in BusStops \
        and stop_pair[1] in BusStops

pair_routes = [(get_route(stop_pair), stop_pair) for stop_pair in stop_pairs if has_stop_location(stop_pair)]
pair_routes = [x for x in pair_routes if x[0] != None]

In [48]:
[a for a in pair_routes if a[1][1] == '14409'], \
[s for s in BusStops.values() if s['BusStopCode'] == '15049'], \
[s for s in BusStops.values() if s['BusStopCode'] == '14419'], \

([],
 [{u'BusStopCode': u'15049',
   u'Description': u'Bef PSA Bldg',
   u'Latitude': 1.274590541032,
   u'Longitude': 103.80216344738955,
   u'RoadName': u'Alexandra Rd',
   u'Source': u'LTA'}],
 [{u'BusStopCode': u'14419',
   u'Description': u'??',
   u'Latitude': u'1.26730338380321',
   u'Longitude': u'103.801017705581',
   u'RoadName': u'LABRADOR VILLA RD',
   u'Source': u'OneMap'}])

In [49]:
# Display a routed path

import folium

m = folium.Map(location=[1.38, 103.8], zoom_start=12)
folium.PolyLine(pair_routes[100][0]).add_to(m)
m

In [53]:
import math

def swap((x,y)):
    return (y,x)

def heading(ll1, ll2):
    xy1 = svy(*swap(ll1))
    xy2 = svy(*swap(ll2))
    
    return (xy2[0] - xy1[0], xy2[1] - xy1[1])

deduped_pair_routes = [
    (dedup_successive(path), stops) for path, stops in pair_routes if len(dedup_successive(path)) >= 2
]

beginning_headings = [
    (heading(path[0], path[1]), stops[0]) for path, stops in deduped_pair_routes
]
end_headings = [
    (heading(path[-2], path[-1]), stops[1]) for path, stops in deduped_pair_routes
]

headings = beginning_headings + end_headings

In [58]:
stop_headings = {
    x: list([z[0] for z in y])
    for x,y in it.groupby(sorted(headings,
                                 key=lambda x:x[1]),
                          key=lambda x:x[1])
}

stop_headings['01029']

[(-136.8867539058847, -179.13235904714384),
 (-136.8867539058847, -179.13235904714384),
 (-136.8867539058847, -179.13235904714384),
 (-4.451601283857599, -5.528779534921341)]

In [59]:
multiple = [(s,x) for s,x in stop_headings.items() if len(x) > 1]

In [62]:
def cosine_distance((a,b), (c,d)):
    return (a*c + b*d) / math.sqrt(a*a + b*b) / math.sqrt(c*c + d*d)

def cosine_distance_to_first(ls):
    
    return [
        cosine_distance(ls[0], l) for l in ls[1:]
    ]

multiple_headings = [ (s, cosine_distance_to_first(l)) for s,l in multiple]

In [67]:
import numpy as np

with_discrepencies = dict([
        (s, similarities)
        for s, similarities in multiple_headings
        if np.min(similarities) < 0.9
    ])
with_discrepencies

{u'01109': [-1.0],
 u'01229': [-1.0000000000000002],
 u'04121': [0.9999999999999998, 0.7349762340801744, 0.7349762340801744],
 u'04251': [0.8686361138393737],
 u'04339': [-0.9999999999999999],
 u'06131': [-1.0, -1.0],
 u'07331': [-0.9998785309252175],
 u'07529': [1.0, 1.0, -1.0],
 u'10121': [-1.0],
 u'10349': [1.0, -1.0],
 u'10589': [-0.999171800368006, -0.999171800368006, -0.999171800368006],
 u'10599': [-0.9999999999999998],
 u'11009': [-1.0000000000000002, 0.9996897182211425],
 u'11379': [0.9999999999999999, -0.04226410568582924],
 u'13099': [-0.9998374178579681, 0.9998542733098286],
 u'13211': [-0.9999222650512156, 0.9963813649085204],
 u'14381': [0.9999999999999998, 0.9999999999999998, -0.9996510346291975],
 u'14399': [-0.9999892649043256, -0.9999990718421152],
 u'16061': [0.8827442476060643, 0.8827442476060643],
 u'16091': [1.0000000000000002, -1.0000000000000002, -1.0000000000000002],
 u'16099': [0.8733555594564414, 0.8733555594564414],
 u'16121': [-0.999999046384699],
 u'16241'

In [70]:
import numpy as np

# Get the final list...
def average_heading(xys):
    acc = np.array([0.0, 0.0])
    
    for xy in xys:
        xya = np.array(xy)
        xy_normalized = xy / np.linalg.norm(xya)
        
        acc += xy_normalized
        
    return math.atan2(*acc)

stop_average_headings = dict([ 
    (s, average_heading(x))
    for s, x in stop_headings.items()
    if s not in with_discrepencies
])
stop_average_headings

{u'46901': 1.0422956626161561,
 u'16079': 1.8291524058451205,
 u'24509': 0.5729413044707918,
 u'55349': -0.7700957816185992,
 u'65091': 0.41616509784646,
 u'21509': 2.928049674413962,
 u'16071': -1.3185044953133365,
 u'21349': -2.873889407846476,
 u'55341': 2.4282771818768234,
 u'22461': -0.20055460564963049,
 u'20051': -1.5486898085378726,
 u'60061': 2.455913215263856,
 u'11541': -1.3227505050200288,
 u'27281': -0.7442048448027369,
 u'26379': 2.989199007891852,
 u'23441': 0.04573342870837219,
 u'77161': 0.6230132892191065,
 u'11381': 2.8637598876857053,
 u'43279': -1.3561963098944059,
 u'92099': -1.9446170367894067,
 u'22469': 2.938223821905799,
 u'05189': -2.6182731837355124,
 u'11389': -0.2980404371780661,
 u'43271': 1.7179891431703174,
 u'02101': -3.047483428731812,
 u'92091': 1.181435217737692,
 u'13021': -1.71334775979604,
 u'24711': 2.317305707419447,
 u'18241': 1.4130689879115175,
 u'51069': 3.012953469282247,
 u'24719': -0.7984655204087577,
 u'92301': -0.4036291816284798,
 u'5

In [78]:
def add_heading(d, h):
    return dict(
        d.items() + [(u'Heading', None if h is None else h / math.pi * 180)]
    )

bus_stops_with_headings = [
    add_heading(s, stop_average_headings.get(s['BusStopCode']))
    for s in BusStops.values()
]
bus_stops_with_headings

[{u'BusStopCode': u'16079',
  u'Description': u'Bef Pasir Panjang PO',
  u'Heading': 1.8291524058451205,
  u'Latitude': 1.29138264862382,
  u'Longitude': 103.77129836184207,
  u'RoadName': u'Pasir Panjang Rd',
  u'Source': u'LTA'},
 {u'BusStopCode': u'55349',
  u'Description': u'Thomson Gr',
  u'Heading': -0.7700957816185992,
  u'Latitude': 1.38479144169995,
  u'Longitude': 103.8364930927339,
  u'RoadName': u'Lentor Dr',
  u'Source': u'LTA'},
 {u'BusStopCode': u'22469',
  u'Description': u'Blk 653B',
  u'Heading': 2.938223821905799,
  u'Latitude': 1.33649062479414,
  u'Longitude': 103.69631873743737,
  u'RoadName': u'Pioneer Rd Nth',
  u'Source': u'LTA'},
 {u'BusStopCode': u'24029',
  u'Description': u"Mazak S'pore",
  u'Heading': 1.4908924124944454,
  u'Latitude': 1.3224709639188,
  u'Longitude': 103.67132658072524,
  u'RoadName': u'Jln Ahmad Ibrahim',
  u'Source': u'LTA'},
 {u'BusStopCode': u'16071',
  u'Description': u'Opp Pasir Panjang PO',
  u'Heading': -1.3185044953133365,
  u'La

In [79]:
json.dump(bus_stops_with_headings, open('bus_stops_with_headings.json', 'w'), indent=2)

In [99]:
def plot_stop_heading(bus_stop):
    c = BusStops[bus_stop]
    
    loc = (c['Latitude'], c['Longitude'])
    m = folium.Map(location=loc, zoom_start=18)

    # Draw the marker
    folium.Marker(location=loc).add_to(m)

    # Draw the line segment showing the direction
    def next_point(ll, h):
        xy = svy(*swap(ll))

        print xy

        x2 = xy[0] + 100 * math.sin(h)
        y2 = xy[1] + 100 * math.cos(h)

        return svy(x2, y2, inverse=True)

    folium.PolyLine([
            loc,
            swap(next_point(loc, stop_average_headings[bus_stop]))
        ]).add_to(m)

    return m


In [102]:
# check the sanity of the calculated stop headings
bus_stop = BusStops.keys()[12]
plot_stop_heading(bus_stop)

(11805.984280249195, 36421.69212710719)
